In [3]:
import requests
import pandas as pd
import bs4
import time

In [25]:
urlAll = "https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm"
response = requests.get(urlAll)
soup = bs4.BeautifulSoup(response.content)
tag = soup.find('tbody', {'class':"lister-list"})

hrefs = [href["href"] for href in tag.select("a")]
urls = ['https://www.imdb.com'+ href + 'reviews?ref_=tt_ql_3' for href in hrefs]
urls = list(set(urls))

In [27]:
def review_collection(url):
    page = requests.get(url)
    sp = bs4.BeautifulSoup(page.text,'html5lib')
    movie_title = sp.find('div',{'class':'subpage_title_block'}).find('a',{'itemprop':'url'}).text
    
    alls = []
    for s in sp.find_all('div',{'class':'review-container'}):
        all1=[]
        all1.append(movie_title)
        
        title = s.find('a',{'class': 'title'})
        rating = s.find('div',{'class':'ipl-ratings-bar'})
        nameWithDate = s.find('div',{'class':'display-name-date'})
        content = s.find('div',{'class':'content'})
        
        if title is not None:
            all1.append(title.text.strip().replace('\n',''))
        else:
            all1.append("Unknown Title")
            
        if nameWithDate is not None:
            nameWithDate = nameWithDate.text.strip().replace('\n','')
            name = nameWithDate[:nameWithDate.index(' ')]
            date = nameWithDate[nameWithDate.index(' ')+1:]
            all1.append(name)
            all1.append(date)
        else:
            all1.append("Unknown Customer")
            all1.append("Unknown Date")
        
        if rating is not None:
            all1.append(rating.text.strip().replace('\n','').replace('/10', ''))
        else:
            all1.append('-1')
            
        if content is not None:
            all1.append(content.text[:-245].strip().replace('\n',''))
        else:
            all1.append("Unknown Content")
        alls.append(all1)
    return alls

results = []
for i in range(len(urls)):
    results.append(review_collection(urls[i]))
    
flatten = lambda l: [item for sublist in l for item in sublist]

df = pd.DataFrame(flatten(results), columns=["Movie", "Title", "CustomerID", "Date", "Rating", "Content"])
df

,Movie,Title,CustomerID,Date,Rating,Content
0,Wrong Turn,A Structural Mess of a Movie,Blk_Ne1907,March 2021,5,"This movie wasn't awful, to me at least. Peopl..."
1,Wrong Turn,"Flawed, but definitely watchable",TheLegendOfThe428,March 2021,6,"First of all, don't listen to all the brainles..."
2,Wrong Turn,Feeling very torn...,travisthegrimm31,January 2021,4,SPOILER-FREEI just got out of seeing this and ...
3,Wrong Turn,Wrong movie not wrong turn,kelvinpalmer12,February 2021,2,I wasn't sure when I saw the trailer but thoug...
4,Wrong Turn,Turn around.. and don't come back!,bentleyfold13,February 2021,1,"Much love for the original Wrong Turn movies, ..."
...,...,...,...,...,...,...
70,Monster Hunter,so much unfair biased hate for a entertaining ...,kxxxxxx30,December 2020,10,So much negative reviews by trolls that didnt ...
71,Monster Hunter,"Call The Film ""A MONSTER OF A MOVIE"" Or ""Jurra...",RADIO-1S_MR-MOVIE-MAD-AMI_104-FM20,December 2020,8,______________________________________________...
72,Monster Hunter,WHY....,beyondj14,December 2020,1,Dear Paul......WHY did you let doobie white ru...
73,Monster Hunter,Pure popcorn,hh-944502,January 2021,9,If you want non-stop action and humans fightin...


In [6]:
# time.sleep(2)

In [7]:
# df.to_csv('Homework07.csv', index = False, header=True)

In [16]:
group = df.groupby('Movie').size()
group

Movie
365 Days                        25
A Serbian Film                  25
Adverse                         16
Another Round                   25
Avengers: Age of Ultron         25
                                ..
Willy's Wonderland              25
Wonder Woman 1984               25
Wrong Turn                      25
Yes Day                         25
Zack Snyder's Justice League    25
Length: 81, dtype: int64